In [19]:
import pandas as pd
df = pd.read_csv('twitter_training.csv')

In [20]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [27]:
print(df.isnull().sum()

2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    326
dtype: int64


In [29]:
df.columns = ["ID", "Category", "Sentiment", "Text"]

In [31]:
df = df.dropna(subset=["Text"])

In [37]:
df.shape

(71655, 4)

In [34]:
X= df['Text']
y= df['Sentiment']

In [35]:
X

,Text
0,I am coming to the borders and I will kill you...
1,im getting on borderlands and i will kill you ...
2,im coming on borderlands and i will murder you...
3,im getting on borderlands 2 and i will murder ...
4,im getting into borderlands and i can murder y...
...,...
74676,Just realized that the Windows partition of my...
74677,Just realized that my Mac window partition is ...
74678,Just realized the windows partition of my Mac ...
74679,Just realized between the windows partition of...


In [36]:
y

,Sentiment
0,Positive
1,Positive
2,Positive
3,Positive
4,Positive
...,...
74676,Positive
74677,Positive
74678,Positive
74679,Positive


In [62]:
unique_values = df['Sentiment'].unique()
print(unique_values)

['Positive' 'Neutral' 'Negative' 'Irrelevant']


In [38]:
import tensorflow as tf

In [40]:
voc_size = 10000

In [41]:
import nltk
import re #regular expression
from nltk.corpus import stopwords

In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [48]:
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()
# corpus=[]
# for i in range (0,len(X)):
#     review = re.sub('[^a-zA-Z]', ' ', df['Text'][i])
#     review = review.lower()
#     review = review.split()

#     review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus.append(review)

KeyError: 19

In [49]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in df.index: #iterate using the index of the current dataframe, not X
    review = re.sub('[^a-zA-Z]', ' ', df.loc[i, 'Text']) #use loc to access data by index and column name
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [51]:
corpus[1]

'im get borderland kill'

In [53]:
from tensorflow.keras.layers import Embedding  # for word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [54]:
onehot_repr=[one_hot(words, voc_size) for words in corpus]

In [55]:
onehot_repr[ 1]

[3381, 2041, 3177, 5870]

In [56]:
max_length = max(len(sentence.split()) for sentence in corpus)

print(f"The maximum length of the corpus is: {max_length}")

The maximum length of the corpus is: 163


In [57]:
msg_length = 170
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen = msg_length)

In [99]:
#creating a model
embedding_vector_features = 300
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = msg_length))
model.add(LSTM(100))
model.add(Dense(4, activation = 'softmax'))
# model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'], run_eagerly=True)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [100]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_numeric = le.fit_transform(y)

In [101]:
import numpy as np
X_final = np.array(embedded_docs)
y_final=np.array(y_numeric)

In [102]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X_final, y_final, test_size= 0.33, random_state  =42)

In [103]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [104]:

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 68s 91ms/step - accuracy: 0.5381 - loss: 1.0784 - val_accuracy: 0.7292 - val_loss: 0.6993
Epoch 2/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 82s 91ms/step - accuracy: 0.8023 - loss: 0.5314 - val_accuracy: 0.7919 - val_loss: 0.5572
Epoch 3/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 68s 91ms/step - accuracy: 0.8673 - loss: 0.3540 - val_accuracy: 0.8128 - val_loss: 0.5232
Epoch 4/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 82s 91ms/step - accuracy: 0.8998 - loss: 0.2674 - val_accuracy: 0.8264 - val_loss: 0.5127
Epoch 5/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 81s 90ms/step - accuracy: 0.9162 - loss: 0.2179 - val_accuracy: 0.8305 - val_loss: 0.5525
Epoch 6/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 83s 91ms/step - accuracy: 0.9326 - loss: 0.1732 - val_accuracy: 0.8372 - val_loss: 0.5596
Epoch 7/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 69s 91ms/step - accuracy: 0.9417 - loss: 0.1495 - val_accuracy: 0.8436 - val_loss: 0.5684
Epoch 8/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 81s 90ms/step - accuracy: 0.9506 - loss: 0.1264 - 

In [105]:
y_pred = model.predict(X_test)

739/739 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step


In [106]:

y_pred = np.where(y_pred>0.6, 1, 0)


array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       ...,
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [111]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8246712056497653

In [113]:
y_test_single = np.argmax(y_test, axis=1)
y_pred_single = np.argmax(y_pred, axis=1)


cm = confusion_matrix(y_test_single, y_pred_single)
cm

array([[3540,  197,  163,  211],
       [ 541, 6227,  206,  255],
       [ 539,  249, 4821,  221],
       [ 699,  275,  287, 5216]])

In [109]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.79      0.83      4111
           1       0.90      0.86      0.88      7229
           2       0.88      0.83      0.85      5830
           3       0.88      0.81      0.84      6477

   micro avg       0.89      0.82      0.85     23647
   macro avg       0.89      0.82      0.85     23647
weighted avg       0.89      0.82      0.85     23647
 samples avg       0.82      0.82      0.82     23647



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
